In [ ]:
import pandas as pd
import numpy as np

Traitement et fonctions

In [46]:
valuco_pre = pd.read_csv("../ValueCo data extract - ENSAE research project.csv")
valuco_pre['period'] = pd.to_datetime(valuco_pre['period'])
valuco_pre = valuco_pre.sort_values(by='period')

names = valuco_pre['name'].unique().tolist()
company_number = len(names)

valuco = valuco_pre.groupby(['name', 'period', 'score_kind'], as_index=False)['consensus'].mean()

In [47]:
def get_quartile_on_data(x):
    return x["consensus"].quantile([0.25, 0.5, 0.75]).tolist()

def get_quartile(score, q):
    if not 0 <= score <= 100:
        raise ValueError("Score must be between 0 and 100")
    if score <= q[0]:
        return 0
    if score <= q[1]:
        return 1
    if score <= q[2]:
        return 2
    return 3

In [48]:
def print_quantil_table(kind1,kind2,periode):
    valuco_temp = valuco[valuco['period'] == periode]

    valuco_1 = valuco_temp[valuco_temp['score_kind'] == kind1]
    q1 = get_quartile_on_data(valuco_1)

    valuco_2 = valuco_temp[valuco_temp['score_kind'] == kind2]
    q2 = get_quartile_on_data(valuco_2)

    # Noms des lignes (index) et des colonnes
    ligne = ['Q1', 'Q2', 'Q3', 'Q4', 'Nodata']

    # Création du DataFrame avec noms et dates
    df = pd.DataFrame(np.zeros((5, 5)), index=ligne, columns=ligne)

    # Nommer les axes
    df.index.name = kind1 + '\\' + kind2 

    for name in names:
        i = 4
        j = 4

        if name in valuco_1['name'].values:
            i = get_quartile(float(valuco_1[valuco_1['name'] == name].iloc[0, 3]),q1)

        if name in valuco_2['name'].values:
            j = get_quartile(float(valuco_2[valuco_2['name'] == name].iloc[0, 3]),q2)

        df.iloc[i,j] += 1

    df2 = df/company_number
    styled_df = df2.style.background_gradient(cmap='RdYlGn', axis=None)

    # Afficher dans un notebook Jupyter
    return styled_df



In [49]:
def print_esg_dataexistance_table(kind,periode):
    valuco_temp = valuco[valuco['period'] == periode]

    valuco_1 = valuco_temp[valuco_temp['score_kind'] == 'ESG']
    q = get_quartile_on_data(valuco_1)

    valuco_2 = valuco_temp[valuco_temp['score_kind'] == kind]

    # Noms des lignes (index) et des colonnes
    ligne = ['Q1', 'Q2', 'Q3', 'Q4', 'Nodata']
    column = ['Data', 'Nodata']

    # Création du DataFrame avec noms et dates
    df = pd.DataFrame(np.zeros((5, 2)), index=ligne, columns=column)

    # Nommer les axes
    df.index.name = 'ESG\\' + kind 

    for name in names:
        i = 4
        j = 1

        if name in valuco_1['name'].values:
            i = get_quartile(float(valuco_1[valuco_1['name'] == name].iloc[0, 3]),q)

        if name in valuco_2['name'].values:
            j = 0

        df.iloc[i,j] += 1

    df2 = df/company_number
    styled_df = df2.style.background_gradient(cmap='RdYlGn', axis=None)

    # Afficher dans un notebook Jupyter
    return styled_df

Analyse

In [50]:
periode = pd.to_datetime('2024-02-01')
print_quantil_table('E','S',periode)

,Q1,Q2,Q3,Q4,Nodata
E\S,,,,,
Q1,0.115702,0.079890,0.027548,0.024793,0.000000
Q2,0.063361,0.074380,0.057851,0.049587,0.000000
Q3,0.055096,0.044077,0.079890,0.068871,0.000000
Q4,0.016529,0.044077,0.079890,0.101928,0.000000
Nodata,0.000000,0.000000,0.000000,0.000000,0.016529
